In [ ]:
### Subject: MA5851 - Data Science Master Class I 
### Author: Hendrik A. Dreyer
### Due Date: 1 December 2019

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 18 06:53:13 2019

@author: driku
"""
# Subject: MA5851 - Data Science Master Clasa I 
# Author: Hendrik A. Dreyer
# Due Date: 1 December 2019

#Importing packages
from selenium import webdriver
import pandas as pd
import re
from urllib.parse import urlparse, ParseResult
import time
from datetime import datetime, timedelta

#Set the initial date from when to start the scraping  - 20 Novemebr 2019
init_date  = datetime(2019, 11, 20)

class HackerNewsCrawler1(): 
    def __init__(self, start_link):
        self.link_to_explore = start_link 
        
        #We'll be hunting for the following info
        self.hn_entries = pd.DataFrame(columns = ['id','link_title', 'web_link'])
        
        #Use the Chrome browser to do the crawling
        self.driver = webdriver.Chrome(executable_path='C:/Python38/chromedriver')   
        
        # Start at page 1
        self.pagecount = 1
        
        # Next one is initially available
        self.next = True
                
    def run(self):
        while self.next:
            # We'll scrape 1 year's worth of top posts
            if self.pagecount >=365:  
                
                # Save each scraped page's info to csv file
                self.save_data_to_file()
                
                # Go get the next one
                self.next = False
            try:
                #We'll be working backwards in time, one day at a time
                # and we'll be (mis)using the page count as the number of days
                # looking backwards
                tmp_date = init_date - timedelta(days=self.pagecount)  
                
                # Format date in yyyy-mm-dd format
                extract_date_str = tmp_date.strftime("%Y-%m-%d")  
                
                # Set the date on the pageto get
                self.driver.get(self.link_to_explore+"?" + "day=" + extract_date_str)
                
                # Writesome logs
                print("Scraping Hacker News for date : " + extract_date_str)
                
                # Wait 31 seconds for the next page to load (as per HN's robots.txt recommendation)
                # The robots.txt file in https://news.ycombinator.com/news is set to 30 secs
                self.driver.implicitly_wait(30)
                time.sleep(30)
                print("Waiting 35 second before scraping next page...")
                
                # Go get the data for the current page
                self.extract_data()      
                
                # Step counter for next page
                self.pagecount = self.pagecount + 1
                 
            except:
                # Something went wrong
                print ("Cannot get the page " + str(self.link_to_explore))
                
                # Don't traverse more pages
                self.next = False
                
                # Throw exception
                raise
                
    def extract_data(self):
        #ids = self.driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
        
        # Harvest the entry ids, we'll be using them as XPath anchors. 
        ids = self.driver.find_elements_by_xpath("//*[contains(@id,'')]")
        comment_ids = []
        
        # Traverse all the ids and clean them up
        for i in ids:
            tmp = i.get_attribute('id').strip()
            
            # Remove underscores "_"
            if (tmp.find('_') == -1):
                
                # Ids should only contain numercial numbers
                if re.match("[0-9/]+$", tmp):
                    
                    # Strip all the white spaces
                    comment_ids.append(tmp.strip())
        
        # Traverse all the comment ids and extract relevant planned data
        for x in comment_ids:
            
            # Save the comment id
            id = x
            
            #Grab the weblink
            web_link =  self.driver.find_elements_by_xpath('//*[@id="'+ x +'"]/td[3]/a')[0].get_attribute("href")
            
            o = urlparse(web_link)
            ParseResult(scheme='http', netloc='example.com', path='/', params='', query='', fragment='')
            web_link_short = o.netloc
            
            # Grab the link title
            link_title_element = self.driver.find_elements_by_xpath('//*[@id="'+ x +'"]/td[3]/a')[0]
            link_title = link_title_element.text

            # Wave to the struct to write to file
            self.hn_entries.loc[len(self.hn_entries)] = [id, link_title, web_link_short]
            
            print("Extracting id: " + id)
            print("Extracting link_title: " + link_title)
            print("Extracting web_link_short: " + web_link_short)
            
    def save_data_to_file(self):
        # Save the dataframe content to a CSV file
        self.hn_entries.to_csv ('hacker_news_1.csv', index = None, header=True)
        
    def close_spider(self):
        self.driver.quit()
        
# Run the main program

# Fritst, create a new Chrome session
driver = webdriver.Chrome(executable_path=r'C:/Python38/chromedriver') 
driver.get('https://news.ycombinator.com/front')

#https://news.ycombinator.com/front?day=2019-11-20

if __name__ == '__main__':
    
    # URL to crawl
    url = 'https://news.ycombinator.com/front'
    try:
        # Run the first crawler to extract the id, link to the posted article and the heading
        hn_crawler1 = HackerNewsCrawler1(url)
        hn_crawler1.run()
        hn_crawler1.close_spider()
        
    except:
        raise
        

Scraping Hacker News for date : 2019-11-19
Waiting 35 second before scraping next page...
Extracting id: 21567022
Extracting link_title: Hacker Publishes 2TB of Data from Cayman National Bank
Extracting web_link_short: twitter.com
Extracting id: 21577156
Extracting link_title: How to recognize AI snake oil [pdf]
Extracting web_link_short: www.cs.princeton.edu
Extracting id: 21572622
Extracting link_title: Tools we used to create a hit HTML5 game on Steam
Extracting web_link_short: www.codecks.io
Extracting id: 21567481
Extracting link_title: How Containers Work: Overlayfs
Extracting web_link_short: jvns.ca
Extracting id: 21567556
Extracting link_title: Sourcetrail, interactive source explorer, is now free and open-source
Extracting web_link_short: www.sourcetrail.com
Extracting id: 21576936
Extracting link_title: How to run your own mail server (2017)
Extracting web_link_short: www.c0ffee.net
Extracting id: 21572827
Extracting link_title: We ‘may’ have discovered a potential remedy for

Extracting id: 21567630
Extracting link_title: Some near-term arm64 hardening patches
Extracting web_link_short: lwn.net
Scraping Hacker News for date : 2019-11-17
Waiting 35 second before scraping next page...
Extracting id: 21558250
Extracting link_title: Writing userspace USB drivers for abandoned devices
Extracting web_link_short: blog.benjojo.co.uk
Extracting id: 21558793
Extracting link_title: The Efficiency-Destroying Magic of Tidying Up
Extracting web_link_short: florentcrivello.com
Extracting id: 21560101
Extracting link_title: Amtrak Could Turn a Profit for the First Time
Extracting web_link_short: www.wbur.org
Extracting id: 21559139
Extracting link_title: Show HN: Open-Source Alternative to Intercom, Drift, Zendesk, FreshChat
Extracting web_link_short: github.com
Extracting id: 21559526
Extracting link_title: “What Alan Kay Got Wrong About Objects”
Extracting web_link_short: blog.metaobject.com
Extracting id: 21551585
Extracting link_title: Instant stone (just add water) (2

Extracting id: 21552923
Extracting link_title: A Go contract design combining the strong points of official draft v1 and v2
Extracting web_link_short: github.com
Scraping Hacker News for date : 2019-11-15
Waiting 35 second before scraping next page...
Extracting id: 21542753
Extracting link_title: The Unparalleled Genius of John von Neumann
Extracting web_link_short: medium.com
Extracting id: 21537528
Extracting link_title: Amazon’s quest for more, cheaper products has resulted in a flea market of fakes
Extracting web_link_short: www.washingtonpost.com
Extracting id: 21544864
Extracting link_title: Pointless work meetings 'really a form of therapy'
Extracting web_link_short: www.bbc.com
Extracting id: 21545493
Extracting link_title: To motivate people, talk about the context of the work to be done
Extracting web_link_short: www.sametab.com
Extracting id: 21546682
Extracting link_title: Jimmy Wales has quietly launched a Facebook rival
Extracting web_link_short: www.zdnet.com
Extracting

Scraping Hacker News for date : 2019-11-13
Waiting 35 second before scraping next page...
Extracting id: 21523780
Extracting link_title: 16-inch MacBook Pro
Extracting web_link_short: www.apple.com
Extracting id: 21530860
Extracting link_title: John Carmack: I’m going to work on artificial general intelligence
Extracting web_link_short: www.facebook.com
Extracting id: 21525592
Extracting link_title: Brave launches 1.0
Extracting web_link_short: brave.com
Extracting id: 21525878
Extracting link_title: AI Clones Your Voice After Listening for 5 Seconds (2018)
Extracting web_link_short: google.github.io
Extracting id: 21522522
Extracting link_title: Developing open-source FPGA tools
Extracting web_link_short: twitter.com
Extracting id: 21521472
Extracting link_title: More Intel speculative execution vulnerabilities
Extracting web_link_short: mdsattacks.com
Extracting id: 21528425
Extracting link_title: Neutrinos lead to unexpected discovery about eigenvectors and eigenvalues
Extracting we

Waiting 35 second before scraping next page...
Extracting id: 21504672
Extracting link_title: Makers, Don't Let Yourself Be Forced into the 'Manager Schedule'
Extracting web_link_short: blog.nuclino.com
Extracting id: 21508542
Extracting link_title: When your data doesn’t fit in memory: the basic techniques
Extracting web_link_short: pythonspeed.com
Extracting id: 21508210
Extracting link_title: The Next 50 Years of Databases (2015)
Extracting web_link_short: www.cs.cmu.edu
Extracting id: 21504862
Extracting link_title: How Not to Die (2007)
Extracting web_link_short: www.paulgraham.com
Extracting id: 21508495
Extracting link_title: David Lynch’s TV commercials (2017)
Extracting web_link_short: lwlies.com
Extracting id: 21493183
Extracting link_title: Chicago book returns surge 240% after city eliminates fines
Extracting web_link_short: abc7chicago.com
Extracting id: 21508339
Extracting link_title: Moving towards a faster web
Extracting web_link_short: blog.chromium.org
Extracting id: 

...

Extracting id: 18558210
Extracting link_title: Strange earthquake waves rippled around Earth
Extracting web_link_short: www.nationalgeographic.com
Extracting id: 18557861
Extracting link_title: NES.css: 8-bit style CSS framework
Extracting web_link_short: bcrikko.github.io
Extracting id: 18556775
Extracting link_title: How we spent two weeks hunting an NFS bug in the Linux kernel
Extracting web_link_short: about.gitlab.com
Extracting id: 18562975
Extracting link_title: When starting school, younger children are more likely to be diagnosed with ADHD
Extracting web_link_short: news.harvard.edu
Extracting id: 18562851
Extracting link_title: Asana raises $50M at a $1.5B valuation
Extracting web_link_short: techcrunch.com
Extracting id: 18559282
Extracting link_title: GraphQL: A Retrospective
Extracting web_link_short: verve.co
Extracting id: 18559359
Extracting link_title: Programs to Read
Extracting web_link_short: wiki.c2.com
Extracting id: 18557273
Extracting link_title: GAN Dissection:

Extracting id: 18538123
Extracting link_title: Open Source is Not About You
Extracting web_link_short: gist.github.com
Extracting id: 18538881
Extracting link_title: I still miss my headphone jack, and I want it back
Extracting web_link_short: www.fastcompany.com
Extracting id: 18532842
Extracting link_title: East German secret police guide for identifying youth subcultures (1985)
Extracting web_link_short: twitter.com
Extracting id: 18539017
Extracting link_title: Paying for Popularity Can Be Fraud
Extracting web_link_short: www.bloomberg.com
Extracting id: 18545973
Extracting link_title: China Expands Research Funding, Luring U.S. Scientists and Students
Extracting web_link_short: www.npr.org
Extracting id: 18539539
Extracting link_title: Firecracker – Lightweight Virtualization for Serverless Computing
Extracting web_link_short: aws.amazon.com
Extracting id: 18540125
Extracting link_title: Show HN: A tool to make a bot that speaks like you, learning from WhatsApp chats
Extracting we

Extracting id: 18527917
Extracting link_title: AI Mistakes Bus-Side Ad for Famous CEO, Charges Her With Jaywalking
Extracting web_link_short: www.caixinglobal.com
Extracting id: 18526499
Extracting link_title: Swathing is going out of fashion, as most farmers desiccate to ripen their crops
Extracting web_link_short: nautil.us
Extracting id: 18525534
Extracting link_title: Vladimir Lukyanov's hydraulic computer
Extracting web_link_short: pruned.blogspot.com
Extracting id: 18528105
Extracting link_title: Two Years, Four Nanodegree Programs, and a New Career
Extracting web_link_short: blog.udacity.com
Extracting id: 18525119
Extracting link_title: Optimization in the context of machine learning
Extracting web_link_short: blog.algorithmia.com
Extracting id: 18526682
Extracting link_title: Researchers Created Fake 'Master' Fingerprints to Unlock Smartphones
Extracting web_link_short: motherboard.vice.com
Extracting id: 18525448
Extracting link_title: Strato Pi – An Industrial Raspberry Pi
E

Extracting id: 18515789
Extracting link_title: Flutter: the good, the bad and the ugly
Extracting web_link_short: medium.com
Extracting id: 18512268
Extracting link_title: The Story Behind the IKEA Photo of Amsterdam
Extracting web_link_short: petapixel.com
Extracting id: 18513249
Extracting link_title: US asks allies to drop Huawei
Extracting web_link_short: www.nzherald.co.nz
Extracting id: 18515565
Extracting link_title: Model View Controller isn't
Extracting web_link_short: beza1e1.tuxen.de
Extracting id: 18518302
Extracting link_title: Time-lapse video of a rocket launch seen from space
Extracting web_link_short: www.syfy.com
Extracting id: 18518807
Extracting link_title: Lisp Machine Inc. K-machine (2001)
Extracting web_link_short: fare.tunes.org
Extracting id: 18512483
Extracting link_title: What a Real Train System Looks Like (2009)
Extracting web_link_short: newworldeconomics.com
Extracting id: 18513600
Extracting link_title: What Did Tommy Read?
Extracting web_link_short: www

Extracting id: 18503591
Extracting link_title: CUDA 10 in Clojure
Extracting web_link_short: dragan.rocks
Extracting id: 18504300
Extracting link_title: My hiring experience as a submarine sonar operator in the Norwegian Navy
Extracting web_link_short: www.brautaset.org
Extracting id: 18486404
Extracting link_title: Flying for Thanksgiving
Extracting web_link_short: bert.org
Extracting id: 18494256
Extracting link_title: How much bandwidth does the L2 have to give, anyway?
Extracting web_link_short: github.com
Extracting id: 18505755
Extracting link_title: I Can Be the Bank: Individual Investors Buy Busted Mortgages
Extracting web_link_short: www.wsj.com
Extracting id: 18505697
Extracting link_title: Fast, Constant-Time Sphere Indexing
Extracting web_link_short: donw.io
Extracting id: 18498258
Extracting link_title: CDC Food Safety Alert: E. Coli Outbreak Linked to Romaine Lettuce
Extracting web_link_short: www.cdc.gov
Extracting id: 18502488
Extracting link_title: Concurrency Glossary